In [2]:
import pandas as pd
from math import floor
from math import ceil
import numpy as np

Layout= np.arange(1,25)

Commands = pd.read_excel("RSM1_SRAH-190813-58R_0B_16CP-add_ctrls.xlsx", index_col = 0)

Concentrations = pd.read_csv("Dilution_Concentrations_SR.csv", index_col = 0)

In [89]:
Y_Vars = Commands.columns[Commands.isna().all()].tolist()

Levels = set()
X_Vars = Commands.drop(Y_Vars, axis =1)

Factor_Vol = 20

[Levels.update(X_Vars[Factor].unique().tolist()) for Factor in X_Vars]

Levels = sorted(Levels)

Summary = pd.concat([X_Vars[Factor].value_counts() for Factor in X_Vars.columns], axis =1).T

Factors = X_Vars.columns

display(Summary)

,-2.3538604947060935,-1.0,0.0,1.0,2.3538604947060935
TPO,1,16,24,16,1
IL3,1,16,24,16,1
GCSF,1,16,24,16,1
UM171,1,16,24,16,1
Pom,1,16,24,16,1


In [9]:
Dead_Vol = 16
Dilution_Conc = pd.read_csv("Dilution_Concentrations_SR_temp.csv", 
                            index_col = 0)

display(Dilution_Conc)

#Check if the Concentration is too low for this experiment
display(((1/Dilution_Conc.iloc[:,1:].div(Dilution_Conc.Source, axis=0))/len(Factors) < 1 ).any(axis=None))

#mass/moles of Factors used  Sum(Vi*Ci = mass/mols)
Volume_Times_Conc = pd.DataFrame(Dilution_Conc.drop(["Source"], axis =1)
                                .values*(Summary*Factor_Vol+Dead_Vol), 
                                columns = Summary.columns,
                                index = Summary.index)

#Volumes of Manual Concentration which should be made
Manual_Volumes = (Volume_Times_Conc.sum(axis = 1)/(Dilution_Conc.iloc[:,len(Levels)]*1.1))
Manual_Concentrations = Volume_Times_Conc.sum(axis = 1)*len(Factors)/Manual_Volumes

Manual_Volumes = Manual_Volumes*1.1**2
Dilution_Amount = (Manual_Volumes)*Manual_Concentrations/Dilution_Conc.Source.tolist()
Dilution_Conc["Manually Diluted Concentration"] = Manual_Concentrations.tolist()

,Source,-2.353860495,-1,0,1,2.353860495
Factors,,,,,,
TPO,50000,0.001498,0.063096,1,15.848932,667.778748
IL3,100000,0.003866,0.094406,1,10.592537,258.646739
GCSF,100000,0.001498,0.063096,1,15.848932,667.778748
UM171,1000000,0.001498,0.063096,1,15.848932,667.778748
Pom,100000000,0.000020,0.010000,1,100.000000,51017.713360


False

In [88]:
Epitube_Vol = 1600

Min_Dilution_Vol = 0.5 
Cereal_Commands = []

#Modify the Layout list to properly represent where the space in the EpMotion 
Spaces = np.append(Layout, Layout[len(Factors):]) #Assumes just a single Factor rack and one source rack

Total_Volume = (Summary*Factor_Vol+Dead_Vol)*1.1

#Multiply all of the Concentrations by the volume including the dead vol and a 10% buffer
Vol_Times_Conc = pd.DataFrame(Dilution_Conc.drop(["Source", "Manually Diluted Concentration"], axis =1)
                                .values*Total_Volume*len(Factors), 
                                columns = Summary.columns,
                                index = Summary.index)

#Find the volume to add from the manual source concentrations get the correct concentrations
Vol_To_Add = Vol_Times_Conc.div(Dilution_Conc["Manually Diluted Concentration"], axis=0)

#Scale up the amount of factor and media used to avoid cereal dilutions 
Below_Cutoff_Values = Vol_To_Add[Vol_To_Add < Min_Dilution_Vol]
Scaled_Total_Vol = Total_Volume.div(Below_Cutoff_Values)*Min_Dilution_Vol
Scaled_Total_Vol = Scaled_Total_Vol[Scaled_Total_Vol < Epitube_Vol]
Vol_To_Add = Vol_To_Add.where(Scaled_Total_Vol.isna(), 0.5)
Total_Volume = Total_Volume.where(Scaled_Total_Vol.isna(),Scaled_Total_Vol.values)

#Scale the rest of the concentrations to have a volume of the Epitube
Below_Cutoff_Values = Vol_To_Add[Vol_To_Add < Min_Dilution_Vol]
Scaled_Vol_Add = Below_Cutoff_Values.div(Total_Volume)*Epitube_Vol
Vol_To_Add = Vol_To_Add.where(Scaled_Vol_Add.isna(), Scaled_Vol_Add)
Total_Volume = Total_Volume.where(Scaled_Vol_Add.isna(),Epitube_Vol)

#Find the number of cereal dilutions required for each factor and make sure we can fit them on the EpMotion
Min_Dilution_Series = pd.Series([Min_Dilution_Vol]*len(Factors), index=Factors)
Number_of_Dilutions = pd.Series(np.ceil(
                                np.log(Min_Dilution_Series.div(Vol_To_Add.min(axis=1)))/
                                np.log(Epitube_Vol/Min_Dilution_Vol)))
if len(Spaces) < Number_of_Dilutions.sum() + len(Factors)*len(Levels) + len(Factors):
    messagebox.showinfo("Error", "There are too many tubes and too little space in the EpMotion required for cereally diluting the samples")
    quit()
    
#Create Dataframes which contains the concentrations and locations of each factor
Rack_Locations = pd.DataFrame(Spaces[:len(Levels)], 
                         index=Summary.index, 
                         columns = ["Manual Dilution"])
Spaces = Spaces[len(Levels):]
Rack_Conc = pd.DataFrame({"Manual Dilution": Dilution_Conc["Manually Diluted Concentration"]})
Rack_Conc = pd.concat([Rack_Conc, 
                       pd.DataFrame(columns = 
                                    range(1,int(Number_of_Dilutions.max())+1))])
Rack_Locations = pd.concat([Rack_Locations, 
                            pd.DataFrame(columns = 
                                         range(1,int(Number_of_Dilutions.max())+1))])

#Add the Dilutions to the above Dataframes and fill out cereal commands
Source_Rack = 1 #An assumption being made is that the source always has to be on the first rack
for j,Factor in enumerate(Number_of_Dilutions.index):
    Source = Rack_Locations.loc[Factor]["Manual Dilution"]
    for Number in range(int(Number_of_Dilutions.loc[Factor])): #Different Top Up Volume for last dilution
        if np.count_nonzero(Spaces == 24) == 2:
            Destination_Rack = 1 
        else: 
            Destination_Rack = 2
        Well_Location = Spaces[0]
        Spaces = Spaces[1:]
        #Cereal_Commands.extend(Fill_Up(Source_Rack, 
        #                               Source, 
        #   #                           Epitube_Vol, 
        #  #                            Min_Dilution_Vol, 
        # #                             Destination_Rack, 
        #  #                            Well_Location))
        Source = Well_Location
        Rack_Locations.at[Factor,Number+1] = Well_Location
        Rack_Conc.at[Factor,Number+1] = Rack_Conc.iloc[j,Number]*(Min_Dilution_Vol/Epitube_Vol)
        
#Find the volume needed from the cereal dilutions
Below_Cutoff_Values = Vol_To_Add[Vol_To_Add < Min_Dilution_Vol] #Need to recalculate after the scale-up
Min_Dilution_Frame = pd.concat([Min_Dilution_Series]*len(Levels),  axis = 1)
Min_Dilution_Frame.columns = Levels
Dilution_Tubes = pd.DataFrame(np.ceil( #Find which tube needs to be used
                              np.log(Min_Dilution_Frame.div(Below_Cutoff_Values))/
                              np.log(Epitube_Vol/Min_Dilution_Vol)))
Dilution_Frame = Min_Dilution_Frame*(Epitube_Vol/(Min_Dilution_Vol*Min_Dilution_Vol)) 
Diluted_Vol_To_Add =  Below_Cutoff_Values*(Dilution_Frame.pow(Dilution_Tubes))

        
        
        
        
        
        
        
        
        
        
        
        
Below_Cutoff_Values = Vol_To_Add[Vol_To_Add < Min_Dilution_Vol]

#Create a Source_Location dataframe so the dilution tubes are properly created.
Source_Locations = pd.DataFrame(columns=Summary.columns, index = Summary.index)
Source_Locations.iloc[:,len(Levels) - 1] = Spaces[:len(Levels)]

for i,Level in enumerate(Levels):
    Reasonable_Vols = Vol_To_Add[Vol_To_Add > 0.5] # You need to recreate this dataframe each time as it is possible that there are previous level might have changed the volumes
    for j,Factor in enumerate(Factors):
        if pd.isna(Reasonable_Vols.loc[Factor][Level]):
            New_Volume = Vol_Times_Conc.loc[Factor][Level]/Dilution_Conc.iloc[j,i+2]
            Vol_To_Add.loc[Factor][Level] = New_Volume
            Total_Volume.iloc[j,i+1] = Total_Volume.iloc[j,i+1] + ceil(New_Volume)*1.1 #This has two benefiets, one adding more liquid to ensure there is enough and also if there is a change for  
            Source_Conc.loc[Factor][Level] = Dilution_Conc.iloc[j,i+2]

# Checks if the more of the manual volume is being used than expected. This should never happen, but its a good check to have
for i,Factor in enumerate(Factors): 
    Original_Source = Source_Conc.loc[Factor]/Dilution_Conc.loc[Factor]["Manually Diluted Concentration"]
    Original_Source_Levels = Original_Source[Original_Source >= 1].index
    Manual_Volume_Used = Vol_To_Add[Original_Source_Levels].sum(axis = 1).loc[Factor]
    if Manual_Volumes[i] < Manual_Volume_Used:
        Manual_Volumes[i] = Manual_Volume_Used*1.1        

#Copy the Dataframe to have the same indexes and factors and layout
df = pd.DataFrame(columns=df.columns, index=df.index)
Source_Locations = pd.DataFrame(columns=df.columns, index = df.index)
display(Source_Locations)
Source_Locations.iloc[:,len(Levels) - 1] = Spaces[:len(Levels)]
display(Source_Locations)
        
#Using the Source Concentration Dataframe finds where the liquid should be coming from and going to
for i,Factor in reversed(list(enumerate(Factors))):
    for j, Level in reversed(list(enumerate(Levels))[:-1]): 
        Source_Conc_Value = Source_Conc.loc[Factor][Level]
        print(Location)
            
#IT seems like it now takes the new volumes into account and so I should be good, do a good run through before verifiying though. 
#Then it should take the location, take the source concentration and then contruct a Arrival and Departure Dataframe
# From those, it should be trivial to construct a command system which uses total VOlume and volume needed to dispense
#seems like a plan 


,Manual Dilution,1,2
TPO,3672.783115,1.14774,NaN
IL3,1422.557064,0.444549,NaN
GCSF,3672.783115,1.14774,NaN
UM171,3672.783115,1.14774,NaN
Pom,280597.423480,87.6867,0.0274021


,-2.3538604947060935,-1.0,0.0,1.0,2.3538604947060935
TPO,10.437875,439.789321,NaN,NaN,NaN
IL3,69.576605,NaN,NaN,NaN,NaN
GCSF,10.437875,439.789321,NaN,NaN,NaN
UM171,10.437875,439.789321,NaN,NaN,NaN
Pom,5.722191,0.912339,91.23391,NaN,NaN


,Source,-2.353860495,-1,0,1,2.353860495,Manually Diluted Concentration
Factors,,,,,,,
TPO,50000,0.001498,0.063096,1,15.848932,667.778748,3672.783115
IL3,100000,0.003866,0.094406,1,10.592537,258.646739,1422.557064
GCSF,100000,0.001498,0.063096,1,15.848932,667.778748,3672.783115
UM171,1000000,0.001498,0.063096,1,15.848932,667.778748,3672.783115
Pom,100000000,0.000020,0.010000,1,100.000000,51017.713360,280597.423480


NameError: name 'Source_Conc' is not defined